# 🤟 ASL Hand Sign Classifier using Transfer Learning

This notebook demonstrates how to build an image classifier for **American Sign Language (ASL)** hand signs using **transfer learning** with MobileNetV2.

ASL is a visual language, and recognizing hand gestures accurately can have real-world applications in accessibility tools, education, and human-computer interaction. Instead of training a model from scratch, we leverage a pre-trained CNN (MobileNetV2) and fine-tune it for our 36-class ASL dataset (26 letters + 10 digits).

### 🧠 What You'll Learn:
- How to load and preprocess image data efficiently using TensorFlow
- How to use data augmentation to improve model generalization
- How to apply transfer learning with MobileNetV2
- How to fine-tune a model using misclassified validation examples
- How to evaluate model performance visually and quantitatively

> 📦 This project uses a preprocessed version of the [Kaggle ASL Alphabet dataset](https://www.kaggle.com/datasets/grassknoted/asl-alphabet), where images are cropped around a pink bounding box to focus on hand signs.

Let's get started!


### 📦 Import Libraries

We begin by importing all necessary libraries for data loading, preprocessing, model building, and training:

- **Standard libraries** like `os`, `json`, and `pathlib` help manage file paths and directory structures.
- **NumPy and Matplotlib** are used for numerical operations and data visualization.
- **TensorFlow & Keras** provide tools to build and train deep learning models, including:
  - `image_dataset_from_directory` for loading image data
  - `MobileNetV2` as our pre-trained feature extractor
  - Callbacks like `EarlyStopping` to prevent overfitting


In [ ]:
import sys
from pathlib import Path

# Add the parent directory to sys.path
project_root = Path("..").resolve()  # Or "." if notebook is in root
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [ ]:
# === Imports ===
import json
import os

import matplotlib.pyplot as plt
import numpy as np

# TensorFlow & Keras
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2

from utils.plotting import plot_training_history_with_annotations

In [ ]:
# Define the dataset path
data_dir = Path.cwd().parent / Path("data/preprocessed")

### 📥 Load & Prepare the Dataset

In this section, we define some key hyperparameters and load our preprocessed ASL image dataset.

- **Batch size** determines how many images the model sees in one training step.
- **Image size** is set to 160×160 pixels to match the input shape expected by MobileNetV2.
- We use TensorFlow's `image_dataset_from_directory()` to load images from folders and automatically label them based on directory names.

We also split the dataset into **80% training** and **20% validation** using the `validation_split` argument, which ensures the model is evaluated on unseen data during training.


In [ ]:
# Set hyperparameters and image dimensions
BATCH_SIZE = 32
IMG_SIZE = (160, 160)   # Target image size for model input
NUM_CLASSES = 36        # 26 alphabets (A–Z) + 10 digits (0–9)

# Load the training dataset from the preprocessed directory
train_dataset = image_dataset_from_directory(
    data_dir,
    shuffle=True,                    # Shuffle for better training
    batch_size=BATCH_SIZE,           # Number of images per batch
    image_size=IMG_SIZE,             # Resize images to this size
    validation_split=0.2,            # Split 20% for validation
    subset='training',               # This is the training portion
    seed=42                          # Seed for reproducibility
)

# Load the validation dataset (same split and parameters)
validation_dataset = image_dataset_from_directory(
    data_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset='validation',            # validation dataset
    seed=42
)


### 🖼️ Visualize Sample Training Images

Before diving into model training, it's a good idea to take a quick look at some sample images from the training dataset. This helps verify that:

- Images are loading correctly
- Labels match the folder structure
- Preprocessing (e.g., resizing) looks as expected

Below, we display few images from the training dataset along with their corresponding class labels.

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### 🧼 Preprocessing & Data Augmentation

Before feeding images into our model, we apply two important types of preprocessing:

#### 1. MobileNetV2 Input Preprocessing
MobileNetV2 expects input pixel values to be scaled to the range **[-1, 1]**.  
We apply the `preprocess_input` function provided by TensorFlow to ensure our data is aligned with how the model was originally trained on ImageNet. This improves performance and training stability.

#### 2. Data Augmentation
To improve generalization and make the model more robust to variations, we apply lightweight data augmentation using the `tf.keras.Sequential` API. This includes:

- **Random horizontal flipping** – to simulate mirrored hand signs
- **Random rotation** – to tolerate minor hand tilts

These augmentations are applied **only during training**, and help the model learn more flexible feature representations without needing additional data.


In [ ]:
# Enable automatic prefetching for performance
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Prefetch the next batch while the current one is being processed
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

# Preprocessing function specific to MobileNetV2
# Scales input pixels to the range [-1, 1], matching what the model expects
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# function for data augmentation
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential([
    tfl.RandomFlip("horizontal"),
    tfl.RandomRotation(0.1),
])
    
    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

### 🏷️ Inspect Sample Labels

Before training, it's useful to inspect a batch of labels to ensure they’re being loaded and encoded correctly.  
In the block below, we print the numeric class labels from the first batch in the training dataset.


In [ ]:
for images, labels in train_dataset.take(1):
    print(labels.numpy())

### 🏗️ Build the Transfer Learning Model

We use **MobileNetV2**, a lightweight convolutional neural network pretrained on ImageNet, as our base model. Since it has already learned to extract powerful visual features, we reuse those layers and focus on training a custom classification head for our ASL task.

#### Key Steps:
- **`include_top=False`** removes the original classification head from MobileNetV2.
- **Base model is frozen**, meaning its weights won't be updated during the initial training phase.
- **Data augmentation** and **preprocessing** are applied directly to the input layer.
- A **Global Average Pooling** layer condenses spatial dimensions, followed by **Dropout** to reduce overfitting.
- The final **Dense layer** has 36 neurons with softmax activation — one for each ASL class (A–Z + 0–9).

This structure allows us to leverage pre-trained knowledge while tailoring the output for our specific classification task.


In [ ]:
# === Build the Transfer Learning Model ===
def asl_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function
    Returns:
    Returns:
        tf.keras.model
    '''
    
    
    input_shape = image_shape + (3,)
        
    base_model = MobileNetV2(input_shape=IMG_SIZE + (3,),
                             include_top=False,
                             weights='imagenet')
    
    # freeze the base model by making it non trainable
    base_model.trainable = False 

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=input_shape) 
    
    # apply data augmentation to the inputs
    x = data_augmentation(inputs)
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # include dropout with probability of 0.2 to avoid overfitting
    x = tf.keras.layers.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tf.keras.layers.Dense(36, activation='softmax')(x)
        
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
# === Compile and Train (Frozen Base) ===
model = asl_model(IMG_SIZE, data_augmentation)

model.summary()

### 🧠 Compile & Train the Model

Once the model architecture is defined, we compile it with:

- **Adam optimizer**: A popular and efficient choice for deep learning models
- **Sparse Categorical Crossentropy**: Appropriate for multi-class classification with integer-labeled targets
- **Accuracy**: As the evaluation metric

We also use **Early Stopping** to monitor validation loss and prevent overfitting. If the model doesn't improve after 5 consecutive epochs, training stops automatically and the best-performing weights are restored.

We train the model for up to **30 epochs**, but early stopping may end training earlier if the validation performance plateaus.


In [ ]:
# Stop training early if validation loss doesn't improve for 5 epochs
early_stop = EarlyStopping(
    patience=5,                   # Number of epochs with no improvement before stopping
    restore_best_weights=True     # Roll back to the best model weights
)

# Compile the model with optimizer, loss function, and metric
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Adam optimizer with a low learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),  # Use for integer-labeled multiclass tasks
    metrics=['accuracy']                                     # Track accuracy during training
)

# Train the model with early stopping
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,                   # Max number of epochs
    callbacks=[early_stop]       # Stop early if no improvement
)

### 📊 Visualize Training & Validation Metrics

After training the model, it's important to visualize how the training and validation accuracy and loss evolved over epochs. This helps us:

- Understand the model's learning progress
- Identify overfitting or underfitting
- Spot the best performing epoch using early stopping

Below, we use a custom helper function `plot_training_history_with_annotations()` to plot:

- Training vs. validation **accuracy**
- Training vs. validation **loss**

In [ ]:
# plot training and validation performance
save_path = Path.cwd().parent / "assets" / "training_plot.png"
plot_training_history_with_annotations(history, title="Training vs Validation Performance", save_path=save_path)

### 🔍 Identify Misclassified Validation Samples

Before fine-tuning the model, we analyze the validation set to identify which examples were misclassified.

By doing this, we can:
- Detect which classes the model struggles with
- Focus training on these “hard” examples
- Improve overall accuracy without needing more data

In the block below:
- We iterate through the validation dataset
- Compare predicted labels to ground truth
- Store the misclassified images and their corresponding predicted and true labels

These hard examples will be used to reweight the training set during fine-tuning.


In [ ]:
# Identify misclassified validation examples for fine-tuning

# Lists to store images and predictions where the model got it wrong
wrong_images = []
wrong_preds = []
wrong_labels = []

# Class names for mapping predicted labels to text
class_names = validation_dataset.class_names

# Go through each batch in the validation set
for images, labels in validation_dataset:
    preds = model.predict(images)                       # Get model predictions
    pred_classes = np.argmax(preds, axis=1)             # Convert softmax scores to class indices
    true_classes = labels.numpy()                       # Get actual labels

    # Compare predictions with ground truth
    for i in range(len(images)):
        if pred_classes[i] != true_classes[i]:
            # Store misclassified image and labels
            wrong_images.append(images[i].numpy().astype("uint8"))
            wrong_preds.append(pred_classes[i])
            wrong_labels.append(true_classes[i])

### 🔧 Prepare Weighted Training Dataset for Fine-Tuning

Now that we've identified misclassified samples from the validation set, we create a **weighted training dataset** that emphasizes these hard examples.

#### Why?
By assigning higher weights to misclassified examples, we:
- Encourage the model to pay more attention to these difficult cases
- Improve learning on underrepresented or confusing classes
- Boost overall validation performance with targeted fine-tuning

#### Steps:
1. **Unbatch the original training set** so we can add custom weights.
2. **Assign a default weight of `1.0`** to all standard training samples.
3. **Assign a higher weight of `2.0`** to misclassified examples.
4. **Concatenate both datasets** and re-batch them for continued training.

This strategy helps the model focus where it previously struggled, without needing to re-train from scratch.


In [ ]:
# STEP 1: Add weights to original train_ds (unbatched)
train_ds_unbatched = train_dataset.unbatch()

def add_default_weight(image, label):
    return image, label, tf.constant(1.0, dtype=tf.float32)

train_ds_weighted = train_ds_unbatched.map(add_default_weight)

# STEP 2: Create a weighted dataset of hard examples
wrong_images = np.array(wrong_images)
wrong_images = np.array(wrong_images).astype(np.float32)
wrong_weights = np.full(len(wrong_labels), 2.0, dtype=np.float32)

hard_ds = tf.data.Dataset.from_tensor_slices((wrong_images, wrong_labels, wrong_weights))

# STEP 3: Combine and batch
combined_train_ds = train_ds_weighted.concatenate(hard_ds)
combined_train_ds = (
    combined_train_ds.shuffle(1000)
                     .cache()
                     .repeat()
                     .batch(32)
                     .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# With our combined training dataset (standard + hard examples), we now resume training to fine-tune the model.
fine_tune_history = model.fit(combined_train_ds, validation_data=validation_dataset, epochs=10, steps_per_epoch=len(train_dataset))

With our combined training dataset (standard + hard examples), we now resume training to fine-tune the model.

In [ ]:
# Let's lool at fine tuning results
save_path = Path.cwd().parent / "assets" / "fine_tuning_plot.png"
plot_training_history_with_annotations(fine_tune_history, title="Fine Tuning Performance", save_path=save_path)

## 📊 Fine-Tuning Results Summary

After applying targeted fine-tuning and weighted sampling on hard-to-predict ASL signs, the model achieved significant improvements in performance.

### ✅ Key Highlights
- 📈 **Best Validation Accuracy**: 96% at **Epoch 9**
- 🔽 **Lowest Validation Loss**: 0.31 at **Epoch 10**
- 🔁 Model was fine-tuned using **MobileNetV2** with **transfer learning**
- 🧠 Boosted generalization by augmenting training data and re-weighting difficult samples
- 🚀 Demonstrated that small, strategic adjustments can lead to **large performance gains**


💡 _This project proves that targeted model tuning and thoughtful data sampling can make a compact model highly effective—even for complex tasks like ASL recognition._
